# Data Download
Request stage & discahrge data from the USGS National Water Information Service, convert to metric, save it to parquet files for later analysis.

**Steps:**
- Download Data
- Combine the multiple requests into a single dataset
- Examine the dataset for missing data
- Save Entire Raw Dataset
- Convert units to metric
- Save Individual Datasets & Metadata as Tables for Analysis
- Test-Read the Files

**Final Output:**
- gzipped JSON of the WaterML for each station (USGS01541200.json.gz): These are cached versions of what the NWIS returns
- a Hydrofunctions Parquet file (WBdata.parquet) of the entire dataset: Not needed, saved for convenience
- Parquet data tables (**WBdata-stage.parquet** & **WBdata-discharge.parquet**) in metric: **These will be used for later analysis**
- Parquet meta-data tables (WBmeta-stage.parquet & WBmeta-discharge.parquet): These contain the original data-quality flags from the USGS

**Notes on Datasets:**
- data are in metric units (stage: m; discharge: m³)
- Each dataset has 140352 rows: 4 years of 15 minute data
- Missing data is marked 'NaN'; metadata for missing data is marked 'hf.missing'
- There is much more missing discharge data than stage data (10.5% for Hyde 01541303 discharge; 0.2% for Bower stage)

In [1]:
import hydrofunctions as hf
print("Hydrofunctions version: ", hf.__version__)

import pandas as pd
pd.set_option('display.max_columns', None) # This changes the default to show all 35 columns of our data when we display it.
print("Pandas version: ", pd.__version__)

import matplotlib
#%matplotlib notebook
# Use the following for JupyterLab instead:
%matplotlib inline

import analysis_functions
print("Analysis_functions version: ", analysis_functions.__version__)

Hydrofunctions version:  0.2.4
Pandas version:  2.3.0
Analysis_functions version:  2025.05.19


# Download Data
We request:
- Six sites along the West Branch of the Susquehanna
- Stage & Discharge
- Four years of 15 minute data
- Each request will be cached in a file: USGS01541200.json.gz
- Running this page again will use cached data

In [2]:
analysis_stations = ['01541200', '01541303', '01542500', '01545500', '01551500']
#The request is too big, so split into many requests & merge later.
start = '2012-10-01'
end = '2016-10-01'

In [3]:
requests = {}
for station in analysis_stations:
    #parameterCd 00065 is stage in feet above datum, 00060 is discharge in cfs.
    requests[station] = hf.NWIS(station,'iv', start, end, parameterCd=['00060','00065'], file=f'USGS{station}.json.gz')

Requested data from https://nwis.waterservices.usgs.gov/nwis/iv/?format=json%2C1.1&sites=01541200&parameterCd=00060%2C00065&startDT=2012-10-01&endDT=2016-10-01
Saving data to USGS01541200.json.gz
Requested data from https://nwis.waterservices.usgs.gov/nwis/iv/?format=json%2C1.1&sites=01541303&parameterCd=00060%2C00065&startDT=2012-10-01&endDT=2016-10-01
Saving data to USGS01541303.json.gz
Requested data from https://nwis.waterservices.usgs.gov/nwis/iv/?format=json%2C1.1&sites=01542500&parameterCd=00060%2C00065&startDT=2012-10-01&endDT=2016-10-01
Saving data to USGS01542500.json.gz
Requested data from https://nwis.waterservices.usgs.gov/nwis/iv/?format=json%2C1.1&sites=01545500&parameterCd=00060%2C00065&startDT=2012-10-01&endDT=2016-10-01
Saving data to USGS01545500.json.gz
Requested data from https://nwis.waterservices.usgs.gov/nwis/iv/?format=json%2C1.1&sites=01551500&parameterCd=00060%2C00065&startDT=2012-10-01&endDT=2016-10-01
Saving data to USGS01551500.json.gz


# Combine the multiple requests into a single dataset
- Merge the tables
- Merge the metadata

In [4]:
all_data = pd.concat([v.df() for k,v in requests.items()], axis=1)
all_data

,USGS:01541200:00060:00000,USGS:01541200:00060:00000_qualifiers,USGS:01541200:00065:00000,USGS:01541200:00065:00000_qualifiers,USGS:01541303:00060:00000,USGS:01541303:00060:00000_qualifiers,USGS:01541303:00065:00000,USGS:01541303:00065:00000_qualifiers,USGS:01542500:00060:00000,USGS:01542500:00060:00000_qualifiers,USGS:01542500:00065:00000,USGS:01542500:00065:00000_qualifiers,USGS:01545500:00060:00000,USGS:01545500:00060:00000_qualifiers,USGS:01545500:00065:00000,USGS:01545500:00065:00000_qualifiers,USGS:01551500:00060:00000,USGS:01551500:00060:00000_qualifiers,USGS:01551500:00065:00000,USGS:01551500:00065:00000_qualifiers
datetimeUTC,,,,,,,,,,,,,,,,,,,,
2012-10-01 04:00:00+00:00,242.0,A,3.59,A,256.0,A,3.25,A,652.0,A,1.82,A,1340.0,A,0.84,A,2950.0,A,1.35,A
2012-10-01 04:15:00+00:00,246.0,A,3.60,A,256.0,A,3.25,A,652.0,A,1.82,A,1340.0,A,0.84,A,2950.0,A,1.35,A
2012-10-01 04:30:00+00:00,242.0,A,3.59,A,256.0,A,3.25,A,652.0,A,1.82,A,1320.0,A,0.82,A,2970.0,A,1.36,A
2012-10-01 04:45:00+00:00,242.0,A,3.59,A,256.0,A,3.25,A,646.0,A,1.81,A,1320.0,A,0.82,A,2970.0,A,1.36,A
2012-10-01 05:00:00+00:00,242.0,A,3.59,A,256.0,A,3.25,A,646.0,A,1.81,A,1310.0,A,0.81,A,2970.0,A,1.36,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-10-02 02:45:00+00:00,239.0,A,3.40,A,317.0,A,3.31,A,943.0,A,2.02,A,935.0,A,0.39,A,1710.0,A,0.66,A
2016-10-02 03:00:00+00:00,239.0,A,3.40,A,317.0,A,3.31,A,951.0,A,2.03,A,935.0,A,0.39,A,1710.0,A,0.66,A
2016-10-02 03:15:00+00:00,239.0,A,3.40,A,317.0,A,3.31,A,951.0,A,2.03,A,925.0,A,0.38,A,1710.0,A,0.66,A


In [5]:
# Combine all of the dictionaries
all_meta = {}
for request in requests.values():
    all_meta.update(request.meta)

all_meta

{'USGS:01541200': {'siteName': 'WB Susquehanna River near Curwensville, PA',
  'siteLatLongSrs': {'srs': 'EPSG:4326',
   'latitude': 40.9614471,
   'longitude': -78.5191906},
  'timeSeries': {'00060': {'variableFreq': '<15 * Minutes>',
    'variableUnit': 'ft3/s',
    'variableDescription': 'Discharge, cubic feet per second',
    'methodID': '121820',
    'methodDescription': ''},
   '00065': {'variableFreq': '<15 * Minutes>',
    'variableUnit': 'ft',
    'variableDescription': 'Gage height, feet',
    'methodID': '121821',
    'methodDescription': ''}}},
 'USGS:01541303': {'siteName': 'West Branch Susquehanna River at Hyde, PA',
  'siteLatLongSrs': {'srs': 'EPSG:4326',
   'latitude': 41.00450158,
   'longitude': -78.4566886},
  'timeSeries': {'00060': {'variableFreq': '<15 * Minutes>',
    'variableUnit': 'ft3/s',
    'variableDescription': 'Discharge, cubic feet per second',
    'methodID': '121823',
    'methodDescription': ''},
   '00065': {'variableFreq': '<15 * Minutes>',
    'v

In [6]:
# Combine the meta data dictionary and the dataframes into a single NWIS object.
WBdata = requests['01541200']
WBdata.meta = all_meta
WBdata._dataframe = all_data
WBdata

USGS:01541200: WB Susquehanna River near Curwensville, PA
    00060: <15 * Minutes>  Discharge, cubic feet per second 
    00065: <15 * Minutes>  Gage height, feet 
USGS:01541303: West Branch Susquehanna River at Hyde, PA
    00060: <15 * Minutes>  Discharge, cubic feet per second 
    00065: <15 * Minutes>  Gage height, feet 
USGS:01542500: WB Susquehanna River at Karthaus, PA
    00060: <15 * Minutes>  Discharge, cubic feet per second 
    00065: <15 * Minutes>  Gage height, feet 
USGS:01545500: West Branch Susquehanna River at Renovo, PA
    00060: <15 * Minutes>  Discharge, cubic feet per second 
    00065: <15 * Minutes>  Gage height, feet 
USGS:01551500: WB Susquehanna River at Williamsport, PA
    00060: <15 * Minutes>  Discharge, cubic feet per second 
    00065: <15 * Minutes>  Gage height, feet 
Start: 2012-10-01 04:00:00+00:00
End:   2016-10-02 03:45:00+00:00

In [7]:
WBdata.df('q').head()

,USGS:01541200:00060:00000,USGS:01541303:00060:00000,USGS:01542500:00060:00000,USGS:01545500:00060:00000,USGS:01551500:00060:00000
datetimeUTC,,,,,
2012-10-01 04:00:00+00:00,242.0,256.0,652.0,1340.0,2950.0
2012-10-01 04:15:00+00:00,246.0,256.0,652.0,1340.0,2950.0
2012-10-01 04:30:00+00:00,242.0,256.0,652.0,1320.0,2970.0
2012-10-01 04:45:00+00:00,242.0,256.0,646.0,1320.0,2970.0
2012-10-01 05:00:00+00:00,242.0,256.0,646.0,1310.0,2970.0


In [8]:
WBdata.df('stage').head()

,USGS:01541200:00065:00000,USGS:01541303:00065:00000,USGS:01542500:00065:00000,USGS:01545500:00065:00000,USGS:01551500:00065:00000
datetimeUTC,,,,,
2012-10-01 04:00:00+00:00,3.59,3.25,1.82,0.84,1.35
2012-10-01 04:15:00+00:00,3.60,3.25,1.82,0.84,1.35
2012-10-01 04:30:00+00:00,3.59,3.25,1.82,0.82,1.36
2012-10-01 04:45:00+00:00,3.59,3.25,1.81,0.82,1.36
2012-10-01 05:00:00+00:00,3.59,3.25,1.81,0.81,1.36


In [9]:
# four years, plus two days (one leap day and an extra day at the end) times 24 hours in a day times four readings an hour
print(f'The maximum possible count is (4 X 365) +2 x 24 x 4 = {((4*365)+2)*24*4}') 
WBdata.df('discharge').describe()

The maximum possible count is (4 X 365) +2 x 24 x 4 = 140352


,USGS:01541200:00060:00000,USGS:01541303:00060:00000,USGS:01542500:00060:00000,USGS:01545500:00060:00000,USGS:01551500:00060:00000
count,139861.000000,125540.000000,135451.000000,128494.000000,127922.000000
mean,590.091634,794.183991,2030.342906,4278.719349,7819.264716
std,694.974962,872.306429,1994.540469,4564.576595,7880.591226
min,40.900000,75.300000,208.000000,325.000000,682.000000
25%,181.000000,212.000000,663.000000,1200.000000,2300.000000
50%,352.000000,501.000000,1420.000000,2990.000000,5720.000000
75%,680.000000,1030.000000,2590.000000,5520.000000,10100.000000
max,5240.000000,9000.000000,16900.000000,49200.000000,73800.000000


# Save Entire Raw Dataset

In [10]:
# This will save the combined dataset as a NWIS parquet file.
WBdata.save('WBdata.parquet')

# This is how to read a NWIS parquet file into a new NWIS object.
test_read = hf.NWIS(file='WBdata.parquet')
test_read.df().head()

Reading data from WBdata.parquet


,USGS:01541200:00060:00000,USGS:01541200:00060:00000_qualifiers,USGS:01541200:00065:00000,USGS:01541200:00065:00000_qualifiers,USGS:01541303:00060:00000,USGS:01541303:00060:00000_qualifiers,USGS:01541303:00065:00000,USGS:01541303:00065:00000_qualifiers,USGS:01542500:00060:00000,USGS:01542500:00060:00000_qualifiers,USGS:01542500:00065:00000,USGS:01542500:00065:00000_qualifiers,USGS:01545500:00060:00000,USGS:01545500:00060:00000_qualifiers,USGS:01545500:00065:00000,USGS:01545500:00065:00000_qualifiers,USGS:01551500:00060:00000,USGS:01551500:00060:00000_qualifiers,USGS:01551500:00065:00000,USGS:01551500:00065:00000_qualifiers
datetimeUTC,,,,,,,,,,,,,,,,,,,,
2012-10-01 04:00:00+00:00,242.0,A,3.59,A,256.0,A,3.25,A,652.0,A,1.82,A,1340.0,A,0.84,A,2950.0,A,1.35,A
2012-10-01 04:15:00+00:00,246.0,A,3.60,A,256.0,A,3.25,A,652.0,A,1.82,A,1340.0,A,0.84,A,2950.0,A,1.35,A
2012-10-01 04:30:00+00:00,242.0,A,3.59,A,256.0,A,3.25,A,652.0,A,1.82,A,1320.0,A,0.82,A,2970.0,A,1.36,A
2012-10-01 04:45:00+00:00,242.0,A,3.59,A,256.0,A,3.25,A,646.0,A,1.81,A,1320.0,A,0.82,A,2970.0,A,1.36,A
2012-10-01 05:00:00+00:00,242.0,A,3.59,A,256.0,A,3.25,A,646.0,A,1.81,A,1310.0,A,0.81,A,2970.0,A,1.36,A


# Convert units to metric
- Stage: feet --> meters
- Discharge: ft³/s --> m³/s


In [11]:
stage_metric = WBdata.df('stage') * 0.3048
q_metric = WBdata.df('discharge') * 0.0283168

# Simplify stage & discharge columns to just the station ID#
- The original column names derive from the system used by the NWIS:
  - original: "USGS:01541200:00060:00000"
  - "USGS" is the agency collecting the data
  - "01541200" is the station ID#
  - "00060" is the parameter code being collected (discharge, in this case)
  - "00000" is the statistic code for the data (instantaneous values in this case)
  - I will use the station ID# as the new column names. Each parameter is in its own file.

In [12]:
# Define a function for selecting only the station ID part of the original names 
# by kceeping the string starting at the 5th character, and ending at the 12th character from the right.
clean_name = lambda x: x[5:-12]

stage_metric.rename(columns=clean_name, inplace=True)
q_metric.rename(columns=clean_name, inplace=True)

print(f"New column names for stage: {stage_metric.columns}")
print(f"New column names for discharge: {q_metric.columns}")

New column names for stage: Index(['01541200', '01541303', '01542500', '01545500', '01551500'], dtype='object')
New column names for discharge: Index(['01541200', '01541303', '01542500', '01545500', '01551500'], dtype='object')


# Save Individual Datasets & Metadata as Tables for Analysis
These can be read into Pandas Dataframes using: `pd.read_parquet('WBdata-stage.parquet')`

In [13]:
stage_metric.to_parquet('WBdata-stage.parquet', compression='gzip')
WBdata.df('stage','flags').to_parquet('WBmeta-stage.parquet', compression='gzip')
q_metric.to_parquet('WBdata-discharge.parquet', compression='gzip')
WBdata.df('discharge','flags').to_parquet('WBmeta-discharge.parquet', compression='gzip')

# Test-Read the Files: Stage data & stage meta

In [14]:
# Read Stage Data
# Works with compression='gzip'
test_read_stage_parq = pd.read_parquet('WBdata-stage.parquet')
test_read_meta_parq = pd.read_parquet('WBmeta-stage.parquet')

# Print a description of the stage data
print(f"Description of Stage values: \n{test_read_stage_parq.describe()}")

# Print a description of the stage metadata
print(f"\n\nDescription of Stage-Meta values: \n")
for column in test_read_meta_parq.columns:
    print(test_read_meta_parq.loc[:,column].value_counts(dropna=False))

Description of Stage values: 
            01541200       01541303       01542500       01545500  \
count  140087.000000  140311.000000  140347.000000  140296.000000   
mean        1.189119       1.264106       0.863949       0.668804   
std         0.239634       0.375343       0.392439       0.545377   
min         0.701040       0.835152       0.280416      -0.115824   
25%         1.039368       0.972312       0.554736       0.243840   
50%         1.158240       1.176528       0.807720       0.591312   
75%         1.295400       1.438656       1.082040       0.938784   
max         2.462784       3.636264       2.654808       3.861816   

            01551500  
count  140302.000000  
mean        0.911531  
std         0.735612  
min        -0.100584  
25%         0.347472  
50%         0.795528  
75%         1.255776  
max         4.849368  


Description of Stage-Meta values: 

USGS:01541200:00065:00000_qualifiers
A             140087
hf.missing       265
Name: count, dtype: int6

In [15]:
# Note that less than 1% of the stage data is missing!
print("Percent missing stage data")
print(f" 01541200: {265/140352:.3%}\n 01541303: {41/140352:.3%}\n 01542500: {5/140352:.3%}\n 01545500: {56/140352:.3%}\n 01551500: {50/140352:.3%}")

Percent missing stage data
 01541200: 0.189%
 01541303: 0.029%
 01542500: 0.004%
 01545500: 0.040%
 01551500: 0.036%


In [16]:
test_read_stage_parq

,01541200,01541303,01542500,01545500,01551500
datetimeUTC,,,,,
2012-10-01 04:00:00+00:00,1.094232,0.990600,0.554736,0.256032,0.411480
2012-10-01 04:15:00+00:00,1.097280,0.990600,0.554736,0.256032,0.411480
2012-10-01 04:30:00+00:00,1.094232,0.990600,0.554736,0.249936,0.414528
2012-10-01 04:45:00+00:00,1.094232,0.990600,0.551688,0.249936,0.414528
2012-10-01 05:00:00+00:00,1.094232,0.990600,0.551688,0.246888,0.414528
...,...,...,...,...,...
2016-10-02 02:45:00+00:00,1.036320,1.008888,0.615696,0.118872,0.201168
2016-10-02 03:00:00+00:00,1.036320,1.008888,0.618744,0.118872,0.201168
2016-10-02 03:15:00+00:00,1.036320,1.008888,0.618744,0.115824,0.201168


In [17]:
test_read_meta_parq

,USGS:01541200:00065:00000_qualifiers,USGS:01541303:00065:00000_qualifiers,USGS:01542500:00065:00000_qualifiers,USGS:01545500:00065:00000_qualifiers,USGS:01551500:00065:00000_qualifiers
datetimeUTC,,,,,
2012-10-01 04:00:00+00:00,A,A,A,A,A
2012-10-01 04:15:00+00:00,A,A,A,A,A
2012-10-01 04:30:00+00:00,A,A,A,A,A
2012-10-01 04:45:00+00:00,A,A,A,A,A
2012-10-01 05:00:00+00:00,A,A,A,A,A
...,...,...,...,...,...
2016-10-02 02:45:00+00:00,A,A,A,A,A
2016-10-02 03:00:00+00:00,A,A,A,A,A
2016-10-02 03:15:00+00:00,A,A,A,A,A


# Test-Read the Files: Discharge data & Discharge meta

In [18]:
# Test-Read the Files: discharge data & discharge meta

# Read Discharge Data
# Works with compression='gzip'
test_read_discharge_parq = pd.read_parquet('WBdata-discharge.parquet')
test_read_Qmeta_parq = pd.read_parquet('WBmeta-discharge.parquet')

# Print a description of the Discharge values:
print(f"Description of Discharge values: \n{test_read_discharge_parq.describe()}")

# Print a description of the Discharge meta-data:
# Note that there is a lot more missing data in the discharge dataset than in the stage dataset.
print(f"\n\nDescription of Discharge-Meta values: \n")
for column in test_read_Qmeta_parq.columns:
    print(test_read_Qmeta_parq.loc[:,column].value_counts(dropna=False))

Description of Discharge values: 
            01541200       01541303       01542500       01545500  \
count  139861.000000  125540.000000  135451.000000  128494.000000   
mean       16.709507      22.488749      57.492814     121.159640   
std        19.679467      24.700927      56.479004     129.254203   
min         1.158157       2.132255       5.889894       9.202960   
25%         5.125341       6.003162      18.774038      33.980160   
50%         9.967514      14.186717      40.209856      84.667232   
75%        19.255424      29.166304      73.340512     156.308736   
max       148.380032     254.851200     478.553920    1393.186560   

            01551500  
count  127922.000000  
mean      221.416555  
std       223.153126  
min        19.312058  
25%        65.128640  
50%       161.972096  
75%       285.999680  
max      2089.779840  


Description of Discharge-Meta values: 

USGS:01541200:00060:00000_qualifiers
A             139861
hf.missing       491
Name: count, dty

In [19]:
# Note that up to 10.5% of the discharge data is missing!
print("Percent missing discharge data")
print(f" 01541200: {491/140352:6.2%}\n 01541303: {14812/140352:6.2%}\n 01542500: {4901/140352:6.2%}\n 01545500: {11858/140352:6.2%}\n 01551500: {12430/140352:6.2%}")

Percent missing discharge data
 01541200:  0.35%
 01541303: 10.55%
 01542500:  3.49%
 01545500:  8.45%
 01551500:  8.86%


In [20]:
test_read_discharge_parq

,01541200,01541303,01542500,01545500,01551500
datetimeUTC,,,,,
2012-10-01 04:00:00+00:00,6.852666,7.249101,18.462554,37.944512,83.534560
2012-10-01 04:15:00+00:00,6.965933,7.249101,18.462554,37.944512,83.534560
2012-10-01 04:30:00+00:00,6.852666,7.249101,18.462554,37.378176,84.100896
2012-10-01 04:45:00+00:00,6.852666,7.249101,18.292653,37.378176,84.100896
2012-10-01 05:00:00+00:00,6.852666,7.249101,18.292653,37.095008,84.100896
...,...,...,...,...,...
2016-10-02 02:45:00+00:00,6.767715,8.976426,26.702742,26.476208,48.421728
2016-10-02 03:00:00+00:00,6.767715,8.976426,26.929277,26.476208,48.421728
2016-10-02 03:15:00+00:00,6.767715,8.976426,26.929277,26.193040,48.421728


In [21]:
test_read_Qmeta_parq

,USGS:01541200:00060:00000_qualifiers,USGS:01541303:00060:00000_qualifiers,USGS:01542500:00060:00000_qualifiers,USGS:01545500:00060:00000_qualifiers,USGS:01551500:00060:00000_qualifiers
datetimeUTC,,,,,
2012-10-01 04:00:00+00:00,A,A,A,A,A
2012-10-01 04:15:00+00:00,A,A,A,A,A
2012-10-01 04:30:00+00:00,A,A,A,A,A
2012-10-01 04:45:00+00:00,A,A,A,A,A
2012-10-01 05:00:00+00:00,A,A,A,A,A
...,...,...,...,...,...
2016-10-02 02:45:00+00:00,A,A,A,A,A
2016-10-02 03:00:00+00:00,A,A,A,A,A
2016-10-02 03:15:00+00:00,A,A,A,A,A
